# Machine Learning 

In [1]:
import pandas as pd
import numpy as np
import time
import machine_learning_helper as machine_learning_helper
import metrics_helper as metrics_helper
import sklearn.neighbors, sklearn.linear_model, sklearn.ensemble, sklearn.naive_bayes
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold, train_test_split, ShuffleSplit
from sklearn import model_selection
from sklearn import ensemble
from xgboost.sklearn import XGBClassifier


## Read .csv files

In [2]:
df_train_users = pd.read_csv("cleaned_train_user.csv")
df_test_users = pd.read_csv("cleaned_test_user.csv")
df_time_mean_user_id = pd.read_csv("time_mean_user_id.csv")
df_time_total_user_id = pd.read_csv("time_total_user_id.csv")
df_total_action_user_id = pd.read_csv("total_action_user_id.csv")

## Construct sessions data frame

In [3]:
df_total_action_user_id.columns = ['id','action']
df_sessions = pd.merge(df_time_mean_user_id, df_time_total_user_id, on='id', how='outer')
df_sessions = pd.merge(df_sessions, df_total_action_user_id, on='id', how='outer')
df_sessions.columns = ['id','time_mean_user','time_total_user','action']
df_sessions.head()

print("X_train has dimension:",df_train_users.shape)
print("X_test has dimension:",df_test_users.shape)

X_train has dimension: (213451, 16)
X_test has dimension: (62096, 15)


## 1. From data frame to matrix : Construct y_train

What we want now for the training is 2 matrices X_train (matrix of relevant features) and y_train (booking dest)

In [4]:
y_labels, label_enc = machine_learning_helper.buildTargetMat(df_train_users)

## 2. From data frame to matrix : Construct X_train & X_test

### Feature engineering.
Add 3 features : 
- time_mean_user
- time_total_user
- total_action_user


In [5]:
df_train_len = df_train_users.shape[0]
df_train = df_train_users.drop(['country_destination'],axis=1)
df_all = pd.concat((df_train_users, df_test_users), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_sessions, on='id', how='left', left_index=True)

In [6]:
X_train, X_test = machine_learning_helper.buildFeatsMat(df_train_users, df_test_users, df_sessions)

## Cross validation
5 folds cross validation, using ndcg as scoring metric.


In [7]:
X_train = X_train[10000:60000]
y_labels = y_labels[10000:60000]
X_test = X_test[10000:60000]

# Split train dataset into 3 folds 
cv = model_selection.KFold(n_splits=3, random_state=None)

## Machine Learning 
First several algorithms are tried, and optimized through Cross validation and Grid Search. 

Our final model is composed of a voting classifier composed of the previous models optimized.

Models that were tried:
- Random Forest with the following parameters:

    - 'max_depth': [ 4, 6, 8]
    - 'n_estimators': [ 50, 100, 150]


- eXtreme Gradient Boosting XCGB:
    - 'max_depth': [6,8,10],
    - 'learning_rate': [0.3],
    - 'n_estimators': [10,15,20,25],
    - 'objective': ['multi:softprob'],
    - 'gamma': [0],
    - 'subsample': [0.5],
    - 'colsample_bytree': [0.5],
    - 'seed': [0]

- Voting classifer:
    - Soft 
    
The metric used is the nDCG.

## Model 1 : RandomForest

Grid Search to find best parameter.

In [8]:
# Define parameters to tune the model
tune_parameters = {'max_depth': [ 4, 6, 8], 'n_estimators': [ 50, 100, 150]}

# Define random forest model
model = ensemble.RandomForestClassifier(warm_start=True)

# Define GridSearch with crossValidation
gridSearchRandomForest = model_selection.GridSearchCV(model, tune_parameters, cv=cv,scoring=metrics_helper.ndcg_scorer, n_jobs=3, verbose = 10)

# Fit model to data
gridSearchRandomForest.fit(X_train, y_labels)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] n_estimators=50, max_depth=4 ....................................
[CV] n_estimators=50, max_depth=4 ....................................
[CV] n_estimators=50, max_depth=4 ....................................
[CV] ........... n_estimators=50, max_depth=4, score=0.758837 -   6.7s
[CV] n_estimators=100, max_depth=4 ...................................
[CV] ........... n_estimators=50, max_depth=4, score=0.807243 -   6.6s
[CV] n_estimators=100, max_depth=4 ...................................
[CV] ........... n_estimators=50, max_depth=4, score=0.802874 -   6.7s
[CV] n_estimators=100, max_depth=4 ...................................


[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:   33.8s


[CV] .......... n_estimators=100, max_depth=4, score=0.758747 -   6.5s
[CV] n_estimators=150, max_depth=4 ...................................
[CV] .......... n_estimators=100, max_depth=4, score=0.800109 -   6.3s
[CV] n_estimators=150, max_depth=4 ...................................
[CV] .......... n_estimators=100, max_depth=4, score=0.807313 -   6.4s
[CV] n_estimators=150, max_depth=4 ...................................
[CV] .......... n_estimators=150, max_depth=4, score=0.758608 -   6.3s
[CV] n_estimators=50, max_depth=6 ....................................


[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:  1.8min


[CV] .......... n_estimators=150, max_depth=4, score=0.807313 -   6.3s
[CV] .......... n_estimators=150, max_depth=4, score=0.802008 -   6.2s
[CV] n_estimators=50, max_depth=6 ....................................
[CV] n_estimators=50, max_depth=6 ....................................
[CV] ........... n_estimators=50, max_depth=6, score=0.805435 -   7.0s
[CV] ........... n_estimators=50, max_depth=6, score=0.760681 -   6.8s
[CV] ........... n_estimators=50, max_depth=6, score=0.807127 -   6.8s
[CV] n_estimators=100, max_depth=6 ...................................
[CV] n_estimators=100, max_depth=6 ...................................
[CV] n_estimators=100, max_depth=6 ...................................


...

[CV] ........... n_estimators=50, max_depth=8, score=0.812423 -   6.1s
[CV] n_estimators=100, max_depth=8 ...................................
[CV] ........... n_estimators=50, max_depth=8, score=0.765160 -   6.1s
[CV] n_estimators=100, max_depth=8 ...................................
[CV] .......... n_estimators=100, max_depth=8, score=0.766011 -   6.3s
[CV] n_estimators=150, max_depth=8 ...................................
[CV] .......... n_estimators=100, max_depth=8, score=0.808071 -   6.0s
[CV] n_estimators=150, max_depth=8 ...................................
[CV] .......... n_estimators=100, max_depth=8, score=0.809802 -   6.1s
[CV] n_estimators=150, max_depth=8 ...................................
[CV] .......... n_estimators=150, max_depth=8, score=0.764560 -   6.6s


[Parallel(n_jobs=3)]: Done  25 out of  27 | elapsed:  5.7min remaining:   27.4s


[CV] .......... n_estimators=150, max_depth=8, score=0.810400 -   6.4s
[CV] .......... n_estimators=150, max_depth=8, score=0.807945 -   6.6s


[Parallel(n_jobs=3)]: Done  27 out of  27 | elapsed:  5.7min finished


GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=True),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'n_estimators': [50, 100, 150], 'max_depth': [4, 6, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(ndcg_score, needs_proba=True, k=5), verbose=10)

In [ ]:
print("Random Forest Best Score found:")
print(gridSearchRandomForest.best_score_)
print("Random Forest Best parameters set found:")
print(gridSearchRandomForest.best_params_)

Random Forest Best Score found:
0.795119516323
Random Forest Best parameters set found:
{'n_estimators': 50, 'max_depth': 8}


## Model 2 : eXtreme Gradient Boosting XCGB

5 folds cross validation, using ndcg as scoring metric.

Grid Search to find best parameter.

In [ ]:
# Define parameters to tune the model
tune_parameters = {
    'max_depth': [6,8,10],
    'learning_rate': [0.3],
    'n_estimators': [10,15,20,25],
    'objective': ['multi:softprob'],
    'gamma': [0],
    'subsample': [0.5],
    'colsample_bytree': [0.5],
    'seed': [0]
}

# Define eXtreme Gradient Boosting model
model = XGBClassifier()

# Define GridSearch with crossValidation
gridSearchXGB = model_selection.GridSearchCV(model, tune_parameters, cv=cv,scoring=metrics_helper.ndcg_scorer, n_jobs=3, verbose = 10)

# Fit model to data
gridSearchXGB.fit(X_train, y_labels)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=10, max_depth=6, seed=0, subsample=0.5 
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=10, max_depth=6, seed=0, subsample=0.5 
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=10, max_depth=6, seed=0, subsample=0.5 
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=10, max_depth=6, seed=0, subsample=0.5, score=0.808574 -   7.2s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=15, max_depth=6, seed=0, subsample=0.5 
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=10, max_depth=6, seed=0, subsample=0.5, score=0.771561 -   7.3s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_es

[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:  1.1min


[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=15, max_depth=6, seed=0, subsample=0.5, score=0.772696 -   7.4s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=20, max_depth=6, seed=0, subsample=0.5 
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=15, max_depth=6, seed=0, subsample=0.5, score=0.811706 -   7.3s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=20, max_depth=6, seed=0, subsample=0.5 
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=15, max_depth=6, seed=0, subsample=0.5, score=0.820233 -   7.5s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=20, max_depth=6, seed=0, subsample=0.5 
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=20, max_depth=6, seed=0, s

[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:  4.2min


[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=20, max_depth=6, seed=0, subsample=0.5, score=0.812808 -   7.8s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=25, max_depth=6, seed=0, subsample=0.5 
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=20, max_depth=6, seed=0, subsample=0.5, score=0.819907 -   7.6s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=25, max_depth=6, seed=0, subsample=0.5 
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=25, max_depth=6, seed=0, subsample=0.5, score=0.812784 -   7.1s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=10, max_depth=8, seed=0, subsample=0.5 
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=25, max_depth=6, seed=0, s

[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:  6.0min


[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=10, max_depth=8, seed=0, subsample=0.5, score=0.772323 -   6.8s
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=10, max_depth=8, seed=0, subsample=0.5, score=0.807176 -   6.8s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=15, max_depth=8, seed=0, subsample=0.5 
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=15, max_depth=8, seed=0, subsample=0.5 
[CV]  learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=10, max_depth=8, seed=0, subsample=0.5, score=0.818599 -   6.6s
[CV] learning_rate=0.3, colsample_bytree=0.5, objective=multi:softprob, gamma=0, n_estimators=15, max_depth=8, seed=0, subsample=0.5 


In [ ]:
print("eXtreme Gradient Boosting Best Score found:")
print(gridSearchXGB.best_score_)
print("eXtreme Gradient Boosting Best set found:")
print(gridSearchXGB.best_params_)

## Voting
Now we are going to vote between the 2 models optimized with their best parameters

In [ ]:
# Create the sub models
estimators = []
model1 = ensemble.RandomForestClassifier(max_depth=gridSearchRandomForest.best_estimator_.max_depth, 
                                         n_estimators=gridSearchRandomForest.best_estimator_.n_estimators)
estimators.append(('random_forest', model1))

model2 = XGBClassifier(max_depth=gridSearchXGB.best_estimator_.max_depth, 
                       learning_rate=gridSearchXGB.best_estimator_.learning_rate,
                      n_estimators= gridSearchXGB.best_estimator_.n_estimators,
                      objective='multi:softprob',
                      subsample=0.5, colsample_bytree=0.5, seed=0)
estimators.append(('xgb', model2))
# Create Voting classifier
finalModel = ensemble.VotingClassifier(estimators,voting='soft')
results = model_selection.cross_val_score(finalModel, X_train, y_labels, cv=cv, scoring = metrics_helper.ndcg_scorer)
print(results.mean())

## Predict countries from model

In [ ]:
finalModel.fit(X_train,y_labels)
y_pred = finalModel.predict_proba(X_test)  
id_test = df_test_users['id']
cts,idsubmission = machine_learning_helper.get5likelycountries(y_pred, id_test)

ctsSubmission = label_enc.inverse_transform(cts)

## Convert to csv for submission

In [ ]:
df_submission = pd.DataFrame(np.column_stack((idsubmission, ctsSubmission)), columns=['id', 'country'])
df_submission.to_csv('submission_country_dest.csv',index=False)

df_submission = pd.DataFrame(np.column_stack((ids, ctsSubmission)), columns=['id', 'country'])
df_submission.to_csv('submission_country_dest.csv',index=False)